##**Downloading Dataset**

In [ ]:
import requests

def read_dataset_from_url(url):
    response = requests.get(url)

    # Ensure request was successful
    if response.status_code != 200:
        raise Exception(f"Failed to download file: {response.status_code}")

    lines = response.text.strip().split("\n")  # Split text by lines

    # Read first line: total flights (N) and total pairings (M)
    flight_count, pairing_count = map(int, lines[0].split())

    # Read crew pairings
    pairings = []
    for line in lines[1:]:  # Skip first line
        data = list(map(int, line.split()))
        cost = data[0]   # First number is cost
        flights = data[2:]  # Flights covered (skip second number)
        pairings.append([cost, len(flights)] + flights)  # Store in list format

    return flight_count, pairing_count, pairings

# Example usage
url = "https://people.brunel.ac.uk/~mastjjb/jeb/orlib/files/sppnw41.txt"
flight_count, pairing_count, pairings = read_dataset_from_url(url)

# Print parsed data
print("Total Flights:", flight_count)
print("Total Pairings:", pairing_count)
print("First 5 Pairings:", pairings[:5])  # Display first 5 pairings


Total Flights: 17
Total Pairings: 197
First 5 Pairings: [[2259, 5, 1, 3, 4, 8, 10], [3309, 4, 1, 3, 4, 11], [4497, 3, 1, 3, 4], [4965, 4, 1, 4, 9, 11], [5961, 3, 1, 4, 9]]


##**Simulated Annealing(SA)**

In [ ]:
import numpy as np

# Compute the cost function with penalty for uncovered flights
def compute_cost(solution, pairings, flight_count, penalty=10000):
    total_cost = 0
    covered_flights = set()

    for i in range(len(solution)):
        if solution[i] == 1:  # If pairing is selected
            total_cost += pairings[i][0]  # Add pairing cost
            covered_flights.update(pairings[i][2:])  # Add covered flights

    # Apply penalty if not all flights are covered
    missing_flights = flight_count - len(covered_flights)
    if missing_flights > 0:
        total_cost += penalty * missing_flights  # Large penalty for missing flights

    return total_cost

# Generate a neighboring solution while ensuring feasibility
def get_neighbor_greedy(solution, pairings, flight_count):
    """
    Generates a neighboring solution using a greedy set cover approach.

    :param solution: Binary vector of the current solution
    :param pairings: List of crew pairings
    :param flight_count: Total number of flights
    :return: A feasible neighboring solution
    """
    neighbor = solution.copy()

    # Find covered flights before modification
    covered_flights = set()
    for i in range(len(neighbor)):
        if neighbor[i] == 1:
            covered_flights.update(pairings[i][2:])

    # Step 1: Randomly remove a pairing
    selected_indices = [i for i, val in enumerate(neighbor) if val == 1]
    if not selected_indices:
        return neighbor  # Avoid empty solutions

    remove_idx = np.random.choice(selected_indices)  # Pick a random pairing to remove
    neighbor[remove_idx] = 0

    # Step 2: Check missing flights
    new_covered_flights = set()
    for i in range(len(neighbor)):
        if neighbor[i] == 1:
            new_covered_flights.update(pairings[i][2:])

    missing_flights = covered_flights - new_covered_flights

    # Step 3: Greedily select the best replacement pairing
    if missing_flights:
        best_replacement = None
        best_effectiveness = float('inf')

        for j in range(len(pairings)):
            if neighbor[j] == 0:  # Only consider unused pairings
                coverage = set(pairings[j][2:]) & missing_flights
                if coverage:
                    cost_effectiveness = pairings[j][0] / len(coverage)  # Greedy ratio
                    if cost_effectiveness < best_effectiveness:
                        best_replacement = j
                        best_effectiveness = cost_effectiveness

        if best_replacement is not None:
            neighbor[best_replacement] = 1  # Add best replacement pairing

    return neighbor


# Simulated Annealing Algorithm
def simulated_annealing(pairings, flight_count, initial_temp=1000, cooling_rate=0.99, min_temp=1e-3, max_iter=10000):
    M = len(pairings)  # Number of available pairings
    solution = np.random.choice([0, 1], size=M)  # Random initial solution
    current_cost = compute_cost(solution, pairings, flight_count)
    T = initial_temp

    for i in range(max_iter):
        neighbor = get_neighbor_greedy(solution, pairings, flight_count)
        neighbor_cost = compute_cost(neighbor, pairings, flight_count)

        delta_E = neighbor_cost - current_cost

        if delta_E < 0 or np.random.rand() < np.exp(-delta_E / T):
            solution, current_cost = neighbor, neighbor_cost

        T *= cooling_rate  # Reduce temperature

        if T < min_temp:
            break

    return solution, current_cost

In [ ]:
# Run Simulated Annealing with flight costs included
solution, current_cost = simulated_annealing(pairings, flight_count)

print("Best Solution (Selected Pairings):", solution)
print("Best Cost (Including Flights):", current_cost)

Best Solution (Selected Pairings): [0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0]
Best Cost (Including Flights): 31497


In [ ]:
x = set()   #Check if all flights have been covered
for i in range(len(solution)):
    if solution[i] == 1:
        print(pairings[i][2:])
        x.update(pairings[i][2:])

if len(x) == flight_count:
  print("------------------------------")
  print("all flights have been covered")



[1, 3, 4, 8, 10]
[2, 7, 11]
[5, 16, 17]
[6]
[9, 13, 14, 17]
[12, 14, 16]
[14, 15]
------------------------------
all flights have been covered


# Check if all flights have been covered

In [ ]:
import numpy as np

# Compute cost function
def compute_cost(solution, pairings, flight_count, penalty=10000):
    total_cost = 0
    covered_flights = set()

    for i in range(len(solution)):
        if solution[i] == 1:
            total_cost += pairings[i][0]  # Add pairing cost
            covered_flights.update(pairings[i][2:])  # Add covered flights

    missing_flights = flight_count - len(covered_flights)
    if missing_flights > 0:
        total_cost += penalty * missing_flights  # Large penalty for missing flights

    return total_cost

# Generate a neighboring solution
def get_neighbor(solution):
    neighbor = solution.copy()
    idx = np.random.randint(len(solution))
    neighbor[idx] = 1 - neighbor[idx]  # Flip 0 ↔ 1
    return neighbor

# Simulated Annealing Algorithm
def simulated_annealing(pairings, flight_count, initial_temp=1000, cooling_rate=0.99, min_temp=1e-3, max_iter=1000):
    M = len(pairings)
    solution = np.random.choice([0, 1], size=M)
    current_cost = compute_cost(solution, pairings, flight_count)
    T = initial_temp

    for i in range(max_iter):
        neighbor = get_neighbor(solution)
        neighbor_cost = compute_cost(neighbor, pairings, flight_count)

        delta_E = neighbor_cost - current_cost

        if delta_E < 0 or np.random.rand() < np.exp(-delta_E / T):
            solution, current_cost = neighbor, neighbor_cost

        T *= cooling_rate  # Reduce temperature

        if T < min_temp:
            break

    return solution, current_cost

# Run SA
best_solution, best_cost = simulated_annealing(pairings, flight_count)
print("Best Solution (Selected Pairings):", best_solution)
print("Best Cost:", best_cost)


Best Solution (Selected Pairings): [0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0]
Best Cost: 25983
